In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('lr_example').getOrCreate()

In [2]:
from pyspark.ml.regression import LinearRegression

In [3]:
data = spark.read.csv('Ecommerce_Customers.csv', inferSchema=True, header=True)

In [4]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [5]:
data.show()

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37.110597442120856|   4.104543202376424| 487.54750486747207|
|riverarebecca@gma...|1414 David Throug...|   

In [6]:
from pyspark.ml.linalg import Vector
from pyspark.ml.feature import VectorAssembler

In [7]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [8]:
assembler = VectorAssembler(inputCols=['Avg Session Length', 'Time on App', 'Time on Website', 'Length of Membership'],
                            outputCol='features')

In [9]:
output = assembler.transform(data)

In [10]:
output.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)
 |-- features: vector (nullable = true)



In [11]:
output.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

In [12]:
final_data = output.select('features', 'Yearly Amount Spent')

In [13]:
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [14]:
train_data, test_data = final_data.randomSplit([.7, .3])

In [15]:
lr = LinearRegression(labelCol='Yearly Amount Spent')

In [16]:
lr_model = lr.fit(train_data)

In [17]:
test_result = lr_model.evaluate(test_data)

In [18]:
test_result.residuals.show()

+-------------------+
|          residuals|
+-------------------+
|  7.749560570177664|
|-1.8329453590177422|
|  8.871677540156156|
| -6.006747266500213|
|-5.5291704157429535|
| 3.6515231660815743|
| -7.984071734514885|
| -9.351219897773262|
|  9.398491066013321|
|-5.1653002330535855|
|  17.91921825300625|
| 2.5962678207068848|
| -5.436433220173512|
|  1.833315473420896|
| -6.265629361200126|
| -4.024910945019485|
|-15.971452498552253|
|-3.5066831743230296|
| 17.108141119741276|
|-27.607984632871307|
+-------------------+
only showing top 20 rows



In [19]:
test_result.rootMeanSquaredError

9.709064873397503

In [20]:
test_result.r2

0.9860210854829443

In [21]:
unlebeled_data = test_data.select('features')

In [22]:
unlebeled_data.show()

+--------------------+
|            features|
+--------------------+
|[29.5324289670579...|
|[30.5743636841713...|
|[30.7377203726281...|
|[30.8364326747734...|
|[30.8794843441274...|
|[31.0472221394875...|
|[31.0613251567161...|
|[31.1280900496166...|
|[31.1695067987115...|
|[31.2681042107507...|
|[31.3123495994443...|
|[31.3584771924370...|
|[31.4252268808548...|
|[31.4459724827577...|
|[31.5147378578019...|
|[31.5171218025062...|
|[31.5741380228732...|
|[31.5761319713222...|
|[31.6005122003032...|
|[31.6739155032749...|
+--------------------+
only showing top 20 rows



In [23]:
predictions = lr_model.transform(unlebeled_data)

In [24]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[29.5324289670579...| 400.8907905024498|
|[30.5743636841713...| 443.8973591170834|
|[30.7377203726281...| 452.9090646560737|
|[30.8364326747734...| 473.5086476934898|
|[30.8794843441274...|495.73577040059763|
|[31.0472221394875...| 388.8458760229398|
|[31.0613251567161...| 495.5395297924165|
|[31.1280900496166...| 566.6039066448279|
|[31.1695067987115...| 417.9580397362795|
|[31.2681042107507...| 428.6358334068775|
|[31.3123495994443...|445.67219977493437|
|[31.3584771924370...| 492.5796826287685|
|[31.4252268808548...| 536.2031518749354|
|[31.4459724827577...| 483.0436494617077|
|[31.5147378578019...|496.07811735766154|
|[31.5171218025062...| 279.9433315954052|
|[31.5741380228732...| 560.3807246591391|
|[31.5761319713222...| 544.7332671636514|
|[31.6005122003032...|462.06471037135566|
|[31.6739155032749...| 503.3330525427525|
+--------------------+------------